# Optimal Sampling pour des réseaux de neurones peu profond (SNN)

On applique ici la théorie de l'optimal sampling dans le cadre des réseaux de neurones peu profond. 


On définit un réseau de neurone peu profond typique $\Phi_\theta : X = \mathbb{R}^n \rightarrow \mathbb{R} $ de taille $m \in \mathbb{N}$ et de paramètre $\theta = (A_1, b_1, A_0, b_0) \in \mathbb{R}^{1 \times m} \times \mathbb{R}^{m \times n}\times \mathbb{R}^m$ par :
\begin{equation}
    \Phi_{(A_1, b_1, A_0, b_0)}(x) = A_1\sigma(A_0x + b_0) + b_1
\end{equation}

## Définition des fonctions 

Dans un premier temps on définit toutes les fonctions utiles à l'application de l'optimal sampling

In [ ]:
# Importation des bibliotheques
import numpy as np
from scipy.special import legendre
from numpy.polynomial import Polynomial
from numpy.polynomial.legendre import Legendre
import matplotlib.pyplot as plt
from numpy.linalg import cond
from scipy import stats
import pandas as pd
import time
from scipy.stats import norm
from scipy.stats import uniform
from scipy.interpolate import interp1d
from scipy.integrate import cumulative_trapezoid
from numpy.linalg import lstsq
import numpy.random as rnd
import torch
from scipy.linalg import sqrtm
from torch.autograd.functional import jacobian
from torch.func import grad
from numpy.polynomial.hermite import hermgauss

In [2]:
# Définition du réseau de neurones simple
class SimpleNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SimpleNN, self).__init__()
        #self.A0 = torch.nn.Parameter(torch.randn(hidden_dim, input_dim))  # Poids
        #self.b0 = torch.nn.Parameter(torch.randn(hidden_dim))  # Biais
        self.activation = torch.sigmoid  # Fonction d'activation σ
        #self.A1 = torch.nn.Parameter(torch.randn(1, hidden_dim))  # Poids 1
        #self.b1 = torch.nn.Parameter(torch.randn(1))  # Biais 1

    #def forward(self, x):
    #    return torch.matmul( self.A1, self.activation(torch.matmul(self.A0, x) + self.b0) ) + self.b1

    def forward(self, x, A0, b0, A1, b1):
        return torch.matmul(A1, self.activation(torch.matmul(A0, x) + b0) ) + b1

In [52]:
def E_ij(n, m, i, j):

    # INPUT : 2 entiers n et m qui représentent la taill de la matrice, et 2 entiers i et j, qui indiquent respectivement une ligne et une colonne
    # OUTPUT : vecteur de base des matrice de taille n x m avec des 0 partout sauf pour le coefficient i, j

    E = np.zeros((n, m))
    E[i, j] = 1
    return torch.tensor(E)


def e_j(n, j):

    # INPUT : 1 entier n qui représente la taill du vecteur, et 1 entier i, qui indique une ligne
    # OUTPUT : vecteur de base des vecteurs de taille n avec des 0 partout sauf pour le coefficient i

    e = np.zeros(n)  
    e[j] = 1 
    return torch.tensor(e)

In [57]:
def calcul_grad_SNN(A0, b0, A1, b1, h):

    # INPUT : un vecteur x, 4 matrices A0, b0, A1, b1. Ces vecteurs ont été utlisé dans le réseau qui a pour sortie A1*sigma(A0x + b0) + b1
    # OUTPUT : la linéarisation du réseau évalué en x

    d = hidden_dim * (input_dim + 2) + 1
    liste_grad = []
    for j in range(hidden_dim):
        liste_grad.append(lambda x, j = j: (torch.sigmoid(torch.matmul(A0, x)) + b0)[j])
    for i in range(input_dim):
        for j in range(hidden_dim):
            Eij = E_ij(input_dim, hidden_dim, i, j)
            liste_grad.append(lambda x, j = j: (torch.sigmoid(torch.matmul(A0 + torch.matmul(Eij, h), x)) + b0)[j])
    for j in range(hidden_dim):
        ej = e_j(hidden_dim, j)
        liste_grad.append(lambda x, j = j: (torch.sigmoid(torch.matmul(A0, x)) + b0 + torch.matmul(ej, h))[j])

    return liste_grad

[tensor(2.0760, dtype=torch.float64), tensor(0.2393, dtype=torch.float64), tensor(0.4542, dtype=torch.float64), tensor(0.4201, dtype=torch.float64), tensor(2.0422, dtype=torch.float64), tensor(0.2058, dtype=torch.float64), tensor(0.4209, dtype=torch.float64), tensor(0.3863, dtype=torch.float64), tensor(3.0116, dtype=torch.float64), tensor(1.1749, dtype=torch.float64), tensor(1.3898, dtype=torch.float64), tensor(1.3557, dtype=torch.float64)]


In [ ]:
def gauss_hermite_integration(phi_i, phi_j, A0, b0, n_quadra):
    
    # INPUT : 2 fonctions phi_i, phi_j, et les matrices A0 et b0 du réseau. n_quadra est le nombre de points de quadrature
    # OUTPUT : le produit scalaire ( phi_i, phi_j ) par quadrature de gauss

    nodes, weights = hermgauss(n_quadra)  # Récupération des points et poids
    integral = np.sum(weights * phi_i(A0 * nodes + b0) * phi_i(A0 * nodes + b0)) / np.sqrt(np.pi)
    return integral


In [5]:
def calcul_gram_SNN(A0, b0, A1, b1):

    # INPUT : les paramètres du réseau A0, b0, A1, b1
    # OUTPUT : la matrice de Gram de la base de l'espace de linéarisation

    # on récupère la base associé au réseau
    fam_gen = calcul_grad_SNN(A0, b0, A1, b1, h)
    d = hidden_dim*(input_dim + 2) + 1
    gram = torch.zeros(d, d)

    # on itère sur chaque élément de la base
    for i in range(d):
        phi_i = fam_gen[i]
        for j in range(d):
            phi_j = fam_gen[j]
            # on calcul le produit scalaire avec la quadrature de gauss :
            gram[i][j] = gauss_hermite_integration(phi_i, phi_j, A0, b0, n_quadra = 20)
    return gram

In [ ]:
def calcul_base_ON(A0, b0, A1, b1):

    # INTPUT : les paramètres du neurones A0, b0, A1, b1
    # OUTPUT : une base orthonormée de la linéarisation obtenue à partir de la famille génératrice de l'espace et de la matrice de Gram

    # on calcule la famille génératrice et la matrie de gram associé à la linéarisation
    fam_gen = calcul_grad_SNN(A0, b0, A1, b1, h)
    gram = calcul_gram_SNN(A0, b0, A1, b1)

    # calcul de la pseudo inverse
    G_inv = torch.linalg.pinv(gram)
    # calcul de la racine de cette pseudo inverse
    G_inv_sqrt = sqrtm(G_inv)
    d_min = torch.linalg.matrix_rank(G_inv)
    d = len(fam_gen)

    base_ON = torch.zeros(d_min)
    # calcul de la base orthonormée
    for d in range(d_min):
        base_ON[d] = sum(G_inv_sqrt[d][j]*fam_gen[j] for j in range(d))

    return base_ON

## Application globale de la théorie de l'optimal sampling pour les SNN

**Remarque :** Contrairement à $\texttt{optimal\_sampling\_approx\_lin}$, on ne fait pas la descente de gradient directement sur $u_t$ mais sur les paramètres $\theta_t$

In [ ]:
# Application globale de la théorie de l'optimal sampling


# paramètres

# initialisation de theta_t pour l'optimal sampling
A0 = torch.randn(hidden_dim, input_dim).double()
b0 = torch.randn(hidden_dim).double()
A1 = torch.randn(1, hidden_dim).double()
b1 = torch.randn(1).double()
theta_t = torch.tensor([A0, b0, A1, b1])


n_vect = 100    # nombre de point à sampler
m_base = 4      # taille de la base V_m
input_dim = 1   # dimension d'entrée du SNN 
hidden_dim = 4  # dimension des paramètres
step_size = 1/9 # step size 
liste_error = []
liste_error_unif = []
L_min = 3.6*10**(-4)
nb_iter_ut = 1000
# liste de step size pour le faire varier durant la descente de gradient
step_size_var = [2/9*(1/(t+1))**0.9 for t in range(nb_iter_ut)] 
for i in range(10):
    step_size_var[i] = 1


proj = 'quasi_proj' # type de projection utilisée 


for t in range(nb_iter_ut):
    # calcul de la projection souhaité
    base_t = calcul_base_ON(A0, b0, A1, b1)
    projection = calcul_proj_os(n_vect, m_base, a, b, theta_t, proj, 1/4)

    # Etude AVEC optimal sampling
    # descente de gradient
    theta_t = theta_t - step_size*projection
    x_os = sequential_conditional_sampling_discretisation(n_vect, m_base, base_t, a, b)
    L_estim = L(theta_t(x_os), fonction_recherchee(x_os))
    liste_error.append(np.abs(L_estim - L_min))


plt.loglog([t for t in range(nb_iter_ut)], liste_error, label = 'L($theta_t$) - $L_{min,M}$ optim')
plt.loglog([t for t in range(nb_iter_ut)], [L_min for t in range(nb_iter_ut)], '--', label = '$L_{min,M}$', color = 'red')
plt.legend()
plt.xlabel('t')
plt.show()

X = np.linspace(a, b, 100)
plt.plot(X, fonction_recherchee(X), label = 'u')
plt.plot(X, theta_t(X), label = 'approximation')
plt.title(f'Approximation avec {proj}')
plt.legend()
plt.show()